In [1]:
import sys
import pytest
sys.path.append('../')

import numpy as np
from numpy.testing import assert_almost_equal, assert_array_equal
from michael import janet
import astropy.units as u

def test_init():
    # Check initiation of correct properties
    gaiaid = 'test'
    ra , dec = 0.0*u.deg, 0.0*u.deg
    output_path = 'data/'
    pipeline = 'eleanor'

    j = janet(gaiaid, ra, dec, output_path, pipeline, verbose=True)

    assert j.gaiaid == gaiaid
    assert j.ra == ra
    assert j.dec == dec
    assert j.output_path == output_path
    assert j.pipeline == pipeline
    assert j.verbose
    assert type(j.void) == dict
    assert j.pl == 'c'

def test_data_call():
    # Check `prepare_data` loads in test data

    # Check `prepare_data` works under different pipeline calls.
    return 0
def test_get_rotation():
    # Assert `get_rotation` calls the correct functions under different
    # pipelines.
    return 0

def test_decode():
    # Assert error message
    gaiaid = 'test'
    ra , dec = 0.0*u.deg, 0.0*u.deg
    output_path = 'data/'
    pipeline = 'eleanor'

    j = janet(gaiaid, ra, dec, output_path, pipeline, verbose=True)

    out = j.decode(32)
    assert type(out) == str

def test_run():
    # Assert functions are called for test data

    # Assert safety is called
    return 0
# Note: janet.view() and janet.validate_rotation() just call functions that are
# tested separately.


2022-10-17 14:32:24.016322: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [2]:
    # Assert error message
    gaiaid = 'test'
    ra , dec = 0.0*u.deg, 0.0*u.deg
    output_path = 'data/'
    pipeline = 'eleanor'

    j = janet(gaiaid, ra, dec, output_path, pipeline, verbose=True)


In [5]:
out = j.decode(22)
type(out)


------ Decoding Overall Period Flag 22 ------
16: The best overal value does not have a peak-to-peak height that exceeds the scatter in the detrended light curve. Proceed with caution.
4: Only 2 of the 3 estimates of rotation agreed with one another to within 1 sigma.
1: None of the 3 estimates agreed with one another to within 1 sigma. The SW estimate is assumed to be the best in this case, if it is available.

No other flags raised. 



NoneType

In [42]:
flag = 3

In [43]:
(type(flag) is not int) and (type(flag) is not float)

False

In [39]:
type(flag)

str